In [ ]:
import numpy as np
import torch
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn
import sys
seaborn.set()

import warnings
warnings.filterwarnings("ignore")

import reservoir
import reckernel
import kuramoto
%load_ext autoreload
%autoreload 2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Data generation

In [ ]:
L = 22 / (2 * np.pi)  # length
N = 100  # space discretization step
dt = 0.25  # time discretization step
N_train = 10000
N_test = 5000
N_init = 1000  # remove the initial points
tend = (N_train + N_test + N_init) * dt

dns = kuramoto.KS(L=L, N=N, dt=dt, tend=tend)
dns.simulate()
u = dns.uu[ninittransients:] / np.sqrt(N)
[u_train, u_test, _] = np.split(u, [N_data_train, N_data_train+N_data_test], axis=0)

In [ ]:
N_plot = 1000
u_plot = u[:N_plot,:]

plt.figure()
plt.imshow(u.T)
plt.colorbar()
plt.grid(b=None)
plt.xlabel(r"$n, \quad t={:}n$".format(dt))
plt.ylabel(r"$x$")
plt.title("Kuramoto-Sivashisky time series")

# Grid search for Reservoir Computing

In [ ]:
u_train_t = torch.from_numpy(u_train).to(device)
u_test_t = torch.from_numpy(u_test).to(device)
pred_horizon = 15
out_train = torch.roll(u_train_t, -pred_horizon, dims=0)
out_test = torch.roll(u_test_t, -pred_horizon, dims=0)

n_res = 1000
initial_state = torch.randn(n_res).to(device) / np.sqrt(n_res)
n_input = 1 # number of time series
input_len, input_dim = u_train_t.shape # time steps t

input_scale_vec = np.arange(start=0.02, stop=0.5, step=0.02)
res_scale_vec = np.arange(start=0.1, stop=1, step=0.1)
err = torch.zeros(len(input_scale_vec), len(res_scale_vec))

for i, input_scale in tqdm(enumerate(input_scale_vec)):
    for j, res_scale in enumerate(res_scale_vec):
        model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, f='erf')
        X = model.forward(u_train_t, initial_state=initial_state).to(device)

        output_w = model.train(X, out_train)
        pred_output = X @ output_w

        Xtest = model.forward(u_test_t, initial_state=initial_state).to(device)
        pred_output_test = Xtest @ output_w

        err[i, j] = torch.norm(pred_output_test - out_test)

In [ ]:
mini = np.argmin(err.numpy())
input_scale_vec = np.arange(start=0.02, stop=0.2, step=0.02)
res_scale_vec = np.arange(start=0.1, stop=1, step=0.1)
width = len(res_scale_vec)
x = mini // width
y = np.mod(mini, width)
print(err[x, y])
optim_input_scale = input_scale_vec[x]
optim_res_scale = res_scale_vec[y]
print(optim_input_scale, optim_res_scale)